In [1]:
import random
def generate_theme_title_and_story(campaign_title: str, insight: str) -> tuple[str, str]:
    title_templates = [
        "The Power of {keyword}",
        "Mastering {keyword} for Growth",
        "Why {keyword} Matters Now",
        "{keyword}: A New Approach",
        "Unlocking {keyword}"
    ]

    insight_keywords = insight.split()
    keyword = random.choice(insight_keywords if insight_keywords else ["change"])
    title = random.choice(title_templates).format(keyword=keyword.capitalize())

    story = f"This theme explores how '{keyword}' relates to {campaign_title.lower()}. We’ll break down why this is crucial and how to apply it practically."
    return title, story
generate_theme_title_and_story("123", "123")

('123: A New Approach',
 "This theme explores how '123' relates to 123. We’ll break down why this is crucial and how to apply it practically.")